In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from delta import *
import logging

GOLDEN_TABLE_PATH = '/opt/data_lake/golden/house_prices'
PREDICTIONS_TABLE_PATH = '/opt/data_lake/golden/house_prices_predictions'
DRUID_GOLDEN_TABLE_PATH = '/opt/data_lake/golden/druid/house_prices.txt'
DRUID_PREDICTIONS_TABLE_PATH = '/opt/data_lake/golden/druid/house_prices_predictions.txt'

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [3]:
# Create Spark Session
builder = (
    SparkSession
    .builder
    .appName("real-estate-etl")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/airflow/.ivy2/cache
The jars for the packages stored in: /home/airflow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-33a21e94-2be8-41b3-a797-3633b0cd7f48;1.0
	confs: [default]


	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central


	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 563ms :: artifacts dl 16ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 from central in [default]
	org.antlr#antlr4;4.7 from central in [default]
	org.antlr#antlr4-runtime;4.7 from central in [default]
	org.glassfish#javax.json;1.0.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     | 

22/07/09 23:01:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
logger.info('Exporting golden house prices for Druid ...')
golden_data = (
    spark
    .read
    .format('delta')
    .load(GOLDEN_TABLE_PATH)
)

golden_data.coalesce(10).write.mode("overwrite").csv(DRUID_GOLDEN_TABLE_PATH, header=False, sep="\t")



2022-07-09 23:01:55,413 [INFO] Exporting golden house prices for Druid ...


In [5]:
logger.info('Exporting model predictions for Druid ...')
predictions_data = (
    spark
    .read
    .format('delta')
    .load(PREDICTIONS_TABLE_PATH)
)

golden_data.coalesce(10).write.mode("overwrite").csv(DRUID_PREDICTIONS_TABLE_PATH, header=False, sep="\t")

2022-07-09 23:02:21,855 [INFO] Exporting model predictions for Druid ...
